In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Finance related operations
from pandas_datareader import data

# Import this to silence a warning when converting data column of a dataframe on the fly
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

## LOAD DATA

In [ ]:
# Load data
df = pd.read_csv(r'C:\Учеба\Диплом\2014_Financial_Data.csv')

# Drop rows with no information
df.dropna(how='all', inplace=True)

## FIRST LOOK AT THE DATASET

In [ ]:
# Get info about dataset
df.info()

# Describe dataset variables
df.describe()

In [ ]:
# Plot class distribution
df_class = df['Class'].value_counts()
sns.barplot(x=df_class.index, y=df_class.values)
plt.title('CLASS COUNT', fontsize=20)
plt.show()

# Plot sector distribution
df_sector = df['Sector'].value_counts()
sns.barplot(x=df_sector.index, y=df_sector.values)
plt.xticks(rotation=90)
plt.title('SECTORS COUNT', fontsize=20)
plt.show()

In [ ]:
# Extract the columns we need in this step from the dataframe
df_ = df.loc[:, ['Sector', '2015 PRICE VAR [%]']]

# Get list of sectors
sector_list = df_['Sector'].unique()

# Plot the percent price variation for each sector
for sector in sector_list:
    
    temp = df_[df_['Sector'] == sector]

    plt.figure(figsize=(30,5))
    plt.plot(temp['2015 PRICE VAR [%]'])
    plt.title(sector.upper(), fontsize=20)
    plt.show()

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Get stocks that increased more than 500%
gain = 500
top_gainers = df[df['2015 PRICE VAR [%]'] >= gain]
top_gainers = top_gainers['2015 PRICE VAR [%]'].sort_values(ascending=False)
print(f'{len(top_gainers)} STOCKS with more than {gain}% gain.')
print()

# Set
date_start = '2015-01-01'
date_end = '2015-12-31'
tickers = top_gainers.index.values.tolist()
ticker_row = []
for ticker in tickers:
    row_value = df[df['2015 PRICE VAR [%]'].index == ticker].iloc[0]['Unnamed: 0']
    ticker_row.append(row_value)
  
for value in ticker_row:
    # Pull daily prices for each ticker from Yahoo Finance
    daily_price = yf.download(value, date_start, date_end)
    
    # Plot prices with volume
    fig, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]})
    
    ax0.plot(daily_price['Adj Close'])
    ax0.set_title(value, fontsize=18)
    ax0.set_ylabel('Daily Adj Close $', fontsize=14)
    ax1.plot(daily_price['Volume'])
    ax1.set_ylabel('Volume', fontsize=14)

    fig.align_ylabels(ax1)
    fig.tight_layout()
    plt.show()

In [ ]:
# Drop those stocks with inorganic gains
inorganic_stocks = tickers[:-2] # all except last 2
df.drop(inorganic_stocks, axis=0, inplace=True)

In [ ]:
# Check again for gain-outliers
df_ = df.loc[:, ['Sector', '2015 PRICE VAR [%]']]
sector_list = df_['Sector'].unique()

for sector in sector_list:
    
    temp = df_[df_['Sector'] == sector] # get all data for one sector

    plt.figure(figsize=(30,5))
    plt.plot(temp['2015 PRICE VAR [%]'])
    plt.title(sector.upper(), fontsize=20)
    plt.show()

## HANDLE MISSING VALUES, 0-VALUES

In [ ]:
# Drop columns relative to classification, we will use them later
class_data = df.loc[:, ['Class', '2015 PRICE VAR [%]']]
df.drop(['Class', '2015 PRICE VAR [%]'], inplace=True, axis=1)

# Plot initial status of data quality in terms of nan-values and zero-values
nan_vals = df.isna().sum()
zero_vals = df.isin([0]).sum()
ind = np.arange(df.shape[1])

plt.figure(figsize=(50,10))

plt.subplot(2,1,1)
plt.title('INITIAL INFORMATION ABOUT DATASET', fontsize=22)
plt.bar(ind, nan_vals.values.tolist())
plt.ylabel('NAN-VALUES COUNT', fontsize=18)

plt.subplot(2,1,2)
plt.bar(ind, zero_vals.values.tolist())
plt.ylabel('ZERO-VALUES COUNT', fontsize=18)
plt.xticks(ind, nan_vals.index.values, rotation=90)

plt.show()

In [ ]:
# Find count and percent of nan-values, zero-values
total_nans = df.isnull().sum().sort_values(ascending=False)
percent_nans = (df.isnull().sum()/df.isnull().count() * 100).sort_values(ascending=False)
total_zeros = df.isin([0]).sum().sort_values(ascending=False)
percent_zeros = (df.isin([0]).sum()/df.isin([0]).count() * 100).sort_values(ascending=False)
df_nans = pd.concat([total_nans, percent_nans], axis=1, keys=['Total NaN', 'Percent NaN'])
df_zeros = pd.concat([total_zeros, percent_zeros], axis=1, keys=['Total Zeros', 'Percent Zeros'])

# Graphical representation
plt.figure(figsize=(15,5))
plt.bar(np.arange(30), df_nans['Percent NaN'].iloc[:30].values.tolist())
plt.xticks(np.arange(30), df_nans['Percent NaN'].iloc[:30].index.values.tolist(), rotation=90)
plt.ylabel('NAN-Dominance [%]', fontsize=18)
plt.grid(alpha=0.3, axis='y')
plt.show()

plt.figure(figsize=(15,5))
plt.bar(np.arange(30), df_zeros['Percent Zeros'].iloc[:30].values.tolist())
plt.xticks(np.arange(30), df_zeros['Percent Zeros'].iloc[:30].index.values.tolist(), rotation=90)
plt.ylabel('ZEROS-Dominance [%]', fontsize=18)
plt.grid(alpha=0.3, axis='y')
plt.show()

In [ ]:
# Find reasonable threshold for nan-values situation
test_nan_level = 0.5
print(df_nans.quantile(test_nan_level))
_, thresh_nan = df_nans.quantile(test_nan_level)

# Find reasonable threshold for zero-values situation
test_zeros_level = 0.6
print(df_zeros.quantile(test_zeros_level))
_, thresh_zeros = df_zeros.quantile(test_zeros_level)

In [ ]:
# Clean dataset applying thresholds for both zero values and NaN values
print(f'INITIAL NUMBER OF VARIABLES: {df.shape[1]}')
print()

df_test1 = df.drop(df_nans[df_nans['Percent NaN'] > thresh_nan].index, axis=1)
print(f'NUMBER OF VARIABLES AFTER NaN THRESHOLD {thresh_nan:.2f}%: {df_test1.shape[1]}')
print()

df_zeros_postnan = df_zeros.drop(df_nans[df_nans['Percent NaN'] > thresh_nan].index, axis=0)
df_test2 = df_test1.drop(df_zeros_postnan[df_zeros_postnan['Percent Zeros'] > thresh_zeros].index, axis=1)
print(f'NUMBER OF VARIABLES AFTER Zeros THRESHOLD {thresh_zeros:.2f}%: {df_test2.shape[1]}')

## CORRELATION MATRIX, CHECK MISSING VALUES AGAIN

In [ ]:
# Exclude non-numeric columns
numeric_columns = df_test2.select_dtypes(include=np.number).columns
df_numeric = df_test2[numeric_columns]

# Plot correlation matrix
fig, ax = plt.subplots(figsize=(20, 15))
sns.heatmap(df_numeric.corr(), annot=False, cmap='YlGnBu', vmin=-1, vmax=1, center=0, ax=ax)
plt.show()

In [ ]:
# New check on nan values
plt.figure(figsize=(50,10))

plt.subplot(2,1,1)
plt.title('INFORMATION ABOUT DATASET - CLEANED NAN + ZEROS', fontsize=22)
plt.bar(np.arange(df_test2.shape[1]), df_test2.isnull().sum())
plt.ylabel('NAN-VALUES COUNT', fontsize=18)

plt.subplot(2,1,2)
plt.bar(np.arange(df_test2.shape[1]), df_test2.isin([0]).sum())
plt.ylabel('ZERO-VALUES COUNT', fontsize=18)
plt.xticks(np.arange(df_test2.shape[1]), df_test2.columns.values, rotation=90)

plt.show()

## HANDLE EXTREME VALUES

In [ ]:
# Analyze dataframe
df_test2.describe()

In [ ]:
# Cut outliers
top_quantiles = df_test2[numeric_columns].quantile(0.97)
outliers_top = (df_test2[numeric_columns] > top_quantiles)

low_quantiles = df_test2[numeric_columns].quantile(0.03)
outliers_low = (df_test2[numeric_columns] < low_quantiles)

df_test2[numeric_columns] = df_test2[numeric_columns].mask(outliers_top, top_quantiles, axis=1)
df_test2[numeric_columns] = df_test2[numeric_columns].mask(outliers_low, low_quantiles, axis=1)

# Take a look at the dataframe post-outliers cut
df_test2.describe()

## FILL MISSING VALUES

In [ ]:
# Replace nan-values with mean value of column, considering each sector individually.
df_test2[numeric_columns] = df_test2.groupby('Sector')[numeric_columns].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_numeric = df_test2[numeric_columns]

# Plot correlation matrix of output dataset
fig, ax = plt.subplots(figsize=(20,15)) 
sns.heatmap(df_numeric.corr(), annot=False, cmap='YlGnBu', vmin=-1, vmax=1, center=0, ax=ax)
plt.show()

## ADD TARGET DATA

In [ ]:
# Add back the classification columns
df_out = df_test2.join(class_data)

# Print information about dataset
df_out.info()
df_out.describe()

## Data Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
df_final = df_out.drop(columns=['Unnamed: 0', '2015 PRICE VAR [%]'])

In [ ]:
sector_dict = {}
for i in df_final['Sector'].drop_duplicates():
        test = df_final[df_final['Sector'] == i].value_counts().sort_index().values
        sector_dict[i] = test[1]/sum(test)

In [ ]:
df_final_copy = df_final

df_final_copy['Sector'] = df_final_copy['Sector'].replace(sector_dict)
sector = df_final_copy['Sector']

df_no_sector = df_final_copy.drop(columns=['Sector'])

In [ ]:
df_to_scale = df_no_sector.drop('Class', axis=1)
y = df_final_copy['Class']

In [ ]:
scaler = RobustScaler()

df_scaled = scaler.fit_transform(df_to_scale)

In [ ]:
df_result = np.hstack((df_scaled, sector.values[:, np.newaxis]))

## Build Model and Predict

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score, confusion_matrix,roc_auc_score,roc_curve,auc,RocCurveDisplay, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(df_result, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42)

In [ ]:
best_params = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}

model = GradientBoostingClassifier(**best_params)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

# 2. Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Adjust 'average' parameter as needed
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# 3. Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

## Regression Model

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
df_final_for_reg = df_out.drop(columns=['Unnamed: 0', 'Class'])

In [ ]:
df_no_sector_for_reg = df_final_for_reg.drop(columns=['Sector'])

In [ ]:
df_to_scale_for_reg = df_no_sector_for_reg.drop('2015 PRICE VAR [%]', axis=1)
y = df_final_for_reg['2015 PRICE VAR [%]']

In [ ]:
scaler = RobustScaler()

df_scaled_for_reg = scaler.fit_transform(df_to_scale_for_reg)

In [ ]:
df_result_for_reg = np.hstack((df_scaled_for_reg, sector.values[:, np.newaxis]))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_result_for_reg, y, test_size=0.25, random_state=42)

In [ ]:
model=GradientBoostingRegressor(max_depth=6)
model.fit(X_train, y_train)
predict=model.predict(X_test)

In [ ]:
print(mean_squared_error(y_test, predict))

In [ ]:
r2_score(y_test, predict)

In [ ]:
y_mean = sum(y_test) / len(y_test)

In [ ]:
SS_tot = sum((y_test.values[i] - y_mean)**2 for i in range(len(y_test)))

In [ ]:
SS_res = sum((predict[i] - y_mean)**2 for i in range(len(y_test)))

In [ ]:
R2 = 1 - (SS_res / SS_tot)
print(f"R²: {R2}")

## More analysys

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_2014 = pd.read_csv(r'C:\Учеба\Диплом\2014_Financial_Data.csv')
df_2015 = pd.read_csv(r'C:\Учеба\Диплом\2015_Financial_Data.csv')
df_2016 = pd.read_csv(r'C:\Учеба\Диплом\2016_Financial_Data.csv')
df_2017 = pd.read_csv(r'C:\Учеба\Диплом\2017_Financial_Data.csv')
df_2018 = pd.read_csv(r'C:\Учеба\Диплом\2018_Financial_Data.csv')

In [ ]:
print("Shape of data for Year 2014 is {}".format(df_2014.shape))
print("Shape of data for Year 2015 is {}".format(df_2015.shape))
print("Shape of data for Year 2016 is {}".format(df_2016.shape))
print("Shape of data for Year 2017 is {}".format(df_2017.shape))
print("Shape of data for Year 2018 is {}".format(df_2018.shape))

In [ ]:
# Data missing information for 2014 year
data_info = pd.concat([pd.DataFrame(df_2014.dtypes).T.rename(index={0: 'column type'}),
                      pd.DataFrame(df_2014.isnull().sum()).T.rename(index={0: 'null values (nb)'}),
                      pd.DataFrame(df_2014.isnull().sum() / df_2014.shape[0] * 100).T.rename(index={0: 'null values (%)'})])

display(data_info)

In [ ]:
# Data missing information for 2015 year
data_info = pd.concat([pd.DataFrame(df_2015.dtypes).T.rename(index={0: 'column type'}),
                      pd.DataFrame(df_2015.isnull().sum()).T.rename(index={0: 'null values (nb)'}),
                      pd.DataFrame(df_2015.isnull().sum() / df_2015.shape[0] * 100).T.rename(index={0: 'null values (%)'})])

display(data_info)

In [ ]:
# Data missing information for 2016 year
data_info = pd.concat([pd.DataFrame(df_2016.dtypes).T.rename(index={0: 'column type'}),
                      pd.DataFrame(df_2016.isnull().sum()).T.rename(index={0: 'null values (nb)'}),
                      pd.DataFrame(df_2016.isnull().sum() / df_2016.shape[0] * 100).T.rename(index={0: 'null values (%)'})])

display(data_info)

In [ ]:
# Data missing information for 2017 year
data_info = pd.concat([pd.DataFrame(df_2017.dtypes).T.rename(index={0: 'column type'}),
                      pd.DataFrame(df_2017.isnull().sum()).T.rename(index={0: 'null values (nb)'}),
                      pd.DataFrame(df_2017.isnull().sum() / df_2017.shape[0] * 100).T.rename(index={0: 'null values (%)'})])

display(data_info)

In [ ]:
# Data missing information for 2018 year
data_info = pd.concat([pd.DataFrame(df_2018.dtypes).T.rename(index={0: 'column type'}),
                      pd.DataFrame(df_2018.isnull().sum()).T.rename(index={0: 'null values (nb)'}),
                      pd.DataFrame(df_2018.isnull().sum() / df_2018.shape[0] * 100).T.rename(index={0: 'null values (%)'})])

display(data_info)

In [ ]:
df_2014.fillna(0,inplace=True)
df_2015.fillna(0,inplace=True)
df_2016.fillna(0,inplace=True)
df_2017.fillna(0,inplace=True)
df_2018.fillna(0,inplace=True)

In [ ]:
# Considering Procter & Gamble company only 
df_2014 = df_2014[df_2014['Unnamed: 0'] =='PG' ]
df_2015 = df_2015[df_2015['Unnamed: 0'] =='PG' ]
df_2016 = df_2016[df_2016['Unnamed: 0'] =='PG' ]
df_2017 = df_2017[df_2017['Unnamed: 0'] =='PG' ]
df_2018 = df_2018[df_2018['Unnamed: 0'] =='PG' ]

In [ ]:
df = pd.concat([df_2014,df_2015,df_2016,df_2017,df_2018])
df.fillna(0,inplace=True)
df.index = [2014,2015,2016,2017,2018]

In [ ]:
print("Mean of Quick Ratio for P&G last 5 year is {:.2f} ".format(df['quickRatio'].mean()))
plt.figure(figsize=(15, 7))
df['quickRatio'].plot.bar(color='g')
plt.xlabel('Years')
plt.ylabel('Quick Ratio')
plt.title('Quick Ratio analysis P&G ')
plt.grid(False)
plt.show()

In [ ]:
# Current ratio
print("Mean of Current Ratio for P&G last 5 year is {:.2f} ".format(df['currentRatio'].mean()))
plt.figure(figsize=(15, 7))
df['currentRatio'].plot.bar()
plt.xlabel('Years')
plt.ylabel('Current Ratio')
plt.title('Current Ratio analysis P&G ')
plt.grid(True)
plt.show()

In [ ]:
# Debt to Equity ratio
print("Mean of Debt to Equity  Ratio for P&G last 5 year is {:.2f} ".format(df['debtEquityRatio'].mean()))
plt.figure(figsize=(15, 7))
df['debtEquityRatio'].plot.bar()
plt.xlabel('Years')
plt.ylabel('debtEquity Ratio')
plt.title('Debt Equity Ratio analysis P&G ')
plt.grid(True)
plt.show()

In [ ]:
# Return on equity
print("Mean of Inventory Turnover for P&G last 5 year is {:.2f} ".format(df['inventoryTurnover'].mean()))
plt.figure(figsize=(15, 7))
df['inventoryTurnover'].plot.bar()
plt.xlabel('Years')
plt.ylabel('Inventory Turnover')
plt.title('Inventory Turnover analysis P&G ')
plt.grid(True)
plt.show()

In [ ]:
df['ShortTermAssest'] = df['Cash and cash equivalents'] + df['Cash and short-term investments']+df['Inventories']+df['Average Receivables']\
                        + df['Investments']+df['Investment purchases and sales']+df['Short-term investments']

df['liquidcash'] = df['Cash and cash equivalents'] + df['Cash and short-term investments']

In [ ]:
# Short Term Assets
n_year = 5
index = np.arange(n_year)
bar_width = 0.35
opacity = 0.7

print("Mean of short Term assest for P&G last 5 year is {:.2f} ".format(df['ShortTermAssest'].mean()))

plt.figure(figsize=(15, 7))
plt.bar(index,df['ShortTermAssest'],bar_width,alpha=opacity,color='b',label='Short Term Assest')
plt.bar(index+bar_width,df['liquidcash'],bar_width,alpha=opacity,color='g',label='liquid Cash')
plt.xlabel('Years')
plt.ylabel('Short Term Assests')
plt.title('Short Term Assest or Current Assest analysis P&G ')
plt.xticks(index+0.20, df.index)
plt.grid(False)
plt.legend()
plt.show()

In [ ]:
# Long Term Asset and Short-Term Asset
df['LongTermAsset'] = df['Property, Plant & Equipment Net'] +df['Goodwill and Intangible Assets']+df['Long-term investments']
# Long term Asset
n_year = 5
index = np.arange(n_year)
bar_width = 0.3
opacity = 0.7

print("Mean of Long Term assest for P&G last 5 year is {:.2f} ".format(df['LongTermAsset'].mean()))

print("Mean Percentage Long Term asset out of Total Asset for P&G last 5 year is {:.2f}% ".format((df['LongTermAsset'].mean()/df['Total assets'].mean())*100))


plt.figure(figsize=(15, 7))
plt.bar(index,df['LongTermAsset'],bar_width,alpha=opacity,color='b',label='Long Term Asset')
plt.bar(index+bar_width,df['ShortTermAssest'],bar_width,alpha=opacity,color='g',label='Short Term Asset')
plt.bar(index-bar_width,df['Total assets'],bar_width,alpha=opacity,color='r',label='Total Asset')
plt.xlabel('Years')
plt.ylabel('Asset Analysis')
plt.title('Short Term Assest and  Long Assest analysis P&G ')
plt.xticks(index+0.10, df.index)
plt.grid(False)
plt.legend()
plt.show();

In [ ]:
# Long Term Asset and Short-Term Asset
df['LongTermAsset'] = df['Property, Plant & Equipment Net'] +df['Goodwill and Intangible Assets']+df['Long-term investments']
# Long term Asset
n_year = 5
index = np.arange(n_year)
bar_width = 0.3
opacity = 0.7

print("Mean of Long Term assest for P&G last 5 year is {:.2f} ".format(df['LongTermAsset'].mean()))

print("Mean Percentage Long Term asset out of Total Asset for P&G last 5 year is {:.2f}% ".format((df['LongTermAsset'].mean()/df['Total assets'].mean())*100))


plt.figure(figsize=(15, 7))
plt.bar(index,df['LongTermAsset'],bar_width,alpha=opacity,color='b',label='Long Term Asset')
plt.bar(index+bar_width,df['ShortTermAssest'],bar_width,alpha=opacity,color='g',label='Short Term Asset')
plt.bar(index-bar_width,df['Total assets'],bar_width,alpha=opacity,color='r',label='Total Asset')
plt.xlabel('Years')
plt.ylabel('Asset Analysis')
plt.title('Short Term Assest and  Long Assest analysis P&G ')
plt.xticks(index+0.10, df.index)
plt.grid(False)
plt.legend()
plt.show();